# [Dataclasses](https://pydantic-docs.helpmanual.io/usage/dataclasses/)

> Note:
>
> These are not in the same order as seen on the docs wiki rather they are
> in alphabetical order as found at `../docs/examples/dataclasses_*.py`

---

If you don't want to use pydantic's `BaseModel` you can instead get the same data
validation on standard [`dataclasses`](https://docs.python.org/3/library/dataclasses.html)
(introduced in python 3.7).

Dataclasses work in python 3.6 using the [`dataclasses backport package`](https://github.com/ericvsmith/dataclasses).

In [7]:
# %load ../docs/examples/dataclasses_main.py
from datetime import datetime
from pydantic.dataclasses import dataclass


@dataclass
class User:
    id: int
    name: str = 'John Doe'
    signup_ts: datetime = None


user = User(id='42', signup_ts='2032-06-21T12:00')
print(user)

User(id=42, name='John Doe', signup_ts=datetime.datetime(2032, 6, 21, 12, 0))


> Note:
>
> Keep in mind that `pydantic.dataclasses.dataclass` is a drop-in replacement for `dataclasses.dataclass` with
> validation, not a replacement for `pydantic.BaseModel` (with a small difference in how [`initialization hooks`](https://pydantic-docs.helpmanual.io/usage/dataclasses/#initialize-hooks) work). 
> There are cases where subclassing pydantic.BaseModel is the better choice.
> 
> For more information and discussion see [`samuelcolvin/pydantic#710`](https://github.com/samuelcolvin/pydantic/issues/710).

You can use all the standard *pydantic* field types, and the resulting dataclass will be identical to the one created by the standard library `dataclass` decorator.

The underlying model and its schema can be accessed through `__pydantic_model__`. Also, fields that require a `default_factory` can be specified by a `dataclasses.field`.

In [3]:
# %load ../docs/examples/dataclasses_default_schema.py
import dataclasses
from typing import List, Optional

from pydantic import Field
from pydantic.dataclasses import dataclass


@dataclass
class User:
    id: int
    name: str = 'John Doe'
    friends: List[int] = dataclasses.field(default_factory=lambda: [0])
    age: Optional[int] = dataclasses.field(
        default=None,
        metadata=dict(title='The age of the user', description='do not lie!')
    )
    height: Optional[int] = Field(None, title='The height in cm', ge=50, le=300)


user = User(id='42')
print(user.__pydantic_model__.schema())

{'title': 'User', 'type': 'object', 'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'title': 'Name', 'default': 'John Doe', 'type': 'string'}, 'friends': {'title': 'Friends', 'type': 'array', 'items': {'type': 'integer'}}, 'age': {'title': 'The age of the user', 'description': 'do not lie!', 'type': 'integer'}, 'height': {'title': 'The height in cm', 'minimum': 50, 'maximum': 300, 'type': 'integer'}}, 'required': ['id']}


In [13]:
# imports ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys
import ruamel.yaml

# YAML dump ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
yml = ruamel.yaml.YAML()
yml.dump(user.__pydantic_model__.schema(), sys.stdout)

title: User
type: object
properties:
  birth:
    $ref: '#/definitions/Birth'
required:
- birth
definitions:
  Birth:
    title: Birth
    type: object
    properties:
      year:
        title: Year
        type: integer
      month:
        title: Month
        type: integer
      day:
        title: Day
        type: integer
    required:
    - year
    - month
    - day


`pydantic.dataclasses.dataclass`'s arguments are the same as the standard decorator, except one extra keyword argument `config` which has the same meaning as [`Config`](https://pydantic-docs.helpmanual.io/usage/model_config/).

> Warning:
>
> After v1.2, `The Mypy plugin` must be installed to type check pydantic dataclasses.

For more information about combining validators with dataclasses, see `dataclass validators`.

---

## Nested dataclasses

Nested `dataclasses` are supported both in dataclasses and normal models.

In [1]:
# %load ../docs/examples/dataclasses_arbitrary_types_allowed.py
import dataclasses

import pydantic


class ArbitraryType:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return f'ArbitraryType(value={self.value!r})'


@dataclasses.dataclass
class DC:
    a: ArbitraryType
    b: str


# valid as it is a builtin dataclass without validation
my_dc = DC(a=ArbitraryType(value=3), b='qwe')

try:
    class Model(pydantic.BaseModel):
        dc: DC
        other: str

    Model(dc=my_dc, other='other')
except RuntimeError as e:  # invalid as it is now a pydantic dataclass
    print(e)


class Model(pydantic.BaseModel):
    dc: DC
    other: str

    class Config:
        arbitrary_types_allowed = True


m = Model(dc=my_dc, other='other')
print(repr(m))


no validator found for <class '__main__.ArbitraryType'>, see `arbitrary_types_allowed` in Config
Model(dc=_Pydantic_DC_94063534265920(a=ArbitraryType(value=3), b='qwe'), other='other')


In [2]:
# imports ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys
import ruamel.yaml

# YAML dump ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
yml = ruamel.yaml.YAML()
#yml.dump(m.dict(), sys.stdout)                         # not working
#yml.dump(m.__pydantic_model__.schema(), sys.stdout)    # not working
print(m)

dc=_Pydantic_DC_94063534265920(a=ArbitraryType(value=3), b='qwe') other='other'


In [5]:
# %load ../docs/examples/dataclasses_initvars.py
from dataclasses import InitVar
from pathlib import Path
from typing import Optional

from pydantic.dataclasses import dataclass


@dataclass
class PathData:
    path: Path
    base_path: InitVar[Optional[Path]]

    def __post_init__(self, base_path):
        print(f'Received path={self.path!r}, base_path={base_path!r}')

    def __post_init_post_parse__(self, base_path):
        if base_path is not None:
            self.path = base_path / self.path


path_data = PathData('world', base_path='/hello')
# Received path='world', base_path='/hello'
assert path_data.path == Path('/hello/world')

Received path='world', base_path='/hello'


In [6]:
# %load ../docs/examples/dataclasses_json_dumps.py
import dataclasses
import json
from typing import List

from pydantic.dataclasses import dataclass
from pydantic.json import pydantic_encoder


@dataclass
class User:
    id: int
    name: str = 'John Doe'
    friends: List[int] = dataclasses.field(default_factory=lambda: [0])


user = User(id='42')
print(json.dumps(user, indent=4, default=pydantic_encoder))

{
    "id": 42,
    "name": "John Doe",
    "friends": [
        0
    ]
}


In [8]:
# %load ../docs/examples/dataclasses_nested.py
from pydantic import AnyUrl
from pydantic.dataclasses import dataclass


@dataclass
class NavbarButton:
    href: AnyUrl


@dataclass
class Navbar:
    button: NavbarButton


navbar = Navbar(button=('https://example.com',))
print(navbar)

Navbar(button=NavbarButton(href=AnyUrl('https://example.com', scheme='https', host='example.com', tld='com', host_type='domain')))


In [9]:
# %load ../docs/examples/dataclasses_post_init_post_parse.py
from pydantic.dataclasses import dataclass


@dataclass
class Birth:
    year: int
    month: int
    day: int


@dataclass
class User:
    birth: Birth

    def __post_init__(self):
        print(self.birth)

    def __post_init_post_parse__(self):
        print(self.birth)


user = User(**{'birth': {'year': 1995, 'month': 3, 'day': 2}})

{'year': 1995, 'month': 3, 'day': 2}
Birth(year=1995, month=3, day=2)


In [10]:
# %load ../docs/examples/dataclasses_stdlib_inheritance.py
import dataclasses

import pydantic


@dataclasses.dataclass
class Z:
    z: int


@dataclasses.dataclass
class Y(Z):
    y: int = 0


@pydantic.dataclasses.dataclass
class X(Y):
    x: int = 0


foo = X(x=b'1', y='2', z='3')
print(foo)

try:
    X(z='pika')
except pydantic.ValidationError as e:
    print(e)

_Pydantic_X_94063534645664(z=3, y=2, x=1)
1 validation error for X
z
  value is not a valid integer (type=type_error.integer)


In [11]:
# %load ../docs/examples/dataclasses_stdlib_to_pydantic.py
import dataclasses
from datetime import datetime
from typing import Optional

import pydantic


@dataclasses.dataclass
class Meta:
    modified_date: Optional[datetime]
    seen_count: int


@dataclasses.dataclass
class File(Meta):
    filename: str


File = pydantic.dataclasses.dataclass(File)

file = File(
    filename=b'thefilename',
    modified_date='2020-01-01T00:00',
    seen_count='7',
)
print(file)

try:
    File(
        filename=['not', 'a', 'string'],
        modified_date=None,
        seen_count=3,
    )
except pydantic.ValidationError as e:
    print(e)

_Pydantic_File_94063534670240(modified_date=datetime.datetime(2020, 1, 1, 0, 0), seen_count=7, filename='thefilename')
1 validation error for File
filename
  str type expected (type=type_error.str)


In [12]:
# %load ../docs/examples/dataclasses_stdlib_with_basemodel.py
import dataclasses
from datetime import datetime
from typing import Optional

from pydantic import BaseModel, ValidationError


@dataclasses.dataclass(frozen=True)
class User:
    name: str


@dataclasses.dataclass
class File:
    filename: str
    last_modification_time: Optional[datetime] = None


class Foo(BaseModel):
    file: File
    user: Optional[User] = None


file = File(
    filename=['not', 'a', 'string'],
    last_modification_time='2020-01-01T00:00',
)  # nothing is validated as expected
print(file)

try:
    Foo(file=file)
except ValidationError as e:
    print(e)

foo = Foo(file=File(filename='myfile'), user=User(name='pika'))
try:
    foo.user.name = 'bulbi'
except dataclasses.FrozenInstanceError as e:
    print(e)

File(filename=['not', 'a', 'string'], last_modification_time='2020-01-01T00:00')
1 validation error for Foo
file -> filename
  str type expected (type=type_error.str)
cannot assign to field 'name'


---